In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x #use tensorflow magic to use version 1.x in colab

TensorFlow 1.x selected.


In [ ]:
#imports
import pandas as pd
import numpy as np
import nltk
import io
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [ ]:
#traning data
df_train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/english_dataset_prepro.csv")
df_train=df_train[df_train.task_2!="NONE"] #remove rows where task2=NONE

In [ ]:
df_train.head()

,text_id,text,task_1,task_2,task_3
1,hasoc_en_2,politico remember clearly individual1 admitted...,HOF,HATE,TIN
7,hasoc_en_8,ados trendingnow blacklivesmatter justice fuck...,HOF,PRFN,TIN
11,hasoc_en_12,don’t know much take 45 compulsive liar trump3...,HOF,HATE,TIN
15,hasoc_en_16,good work icc keep going destroy whole fucking...,HOF,PRFN,TIN
23,hasoc_en_24,shameonicc 1 icc dhoni gloves vs 2 icc plannin...,HOF,HATE,TIN


In [ ]:
Column_Sequence=np.array(df_train["text"],dtype="str") #storing value  of tweets or text in numpy array of strings

In [ ]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+") #create a reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)

In [ ]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()                        #lower case
    sen=Tokenizer.tokenize(sen)           #tokenize the sentences.
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)                #append all in one.

In [ ]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))                           #fit on training text
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300
word_index = tokenizer.word_index      #get a list of all tokens or words in the vocab

In [ ]:
#extract word2vec vectors for words in vocab
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
#load word2vec vectors from file.
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/minor/wordtovec/GoogleNews-vectors-negative300.bin.gz', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector            #store the vector if found.
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)  #store a rndom vector if not found

del(word_vectors)  #del word2vec vectors to free memory

from keras.layers import Embedding
#create an embediing layer from word2vec vectors of our vocab
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
len(tokenizer.word_counts)

9499

In [ ]:
X_train=X_t      #training text

In [ ]:
X_train.shape

(2261, 300)

In [ ]:
Y=df_train["task_2"]   #labels of classes with strings.

In [ ]:
Y = Y.map({'PRFN':0, 'OFFN': 1,'HATE':2})  #map the class names to numbers.

In [ ]:
# One-hot encode label
Y_train=np_utils.to_categorical(Y)

In [ ]:
Y_train.shape

(2261, 3)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.10, random_state=42) #training validation split

In [ ]:
y_train.shape

(2034, 3)

In [ ]:
x_train.shape

(2034, 300)

In [ ]:
max_len=300   #length of text sequence

In [ ]:
x_train

array([[5201,  253,  592, ...,    0,    0,    0],
       [   6,   15, 1828, ...,    0,    0,    0],
       [1091, 6275, 1519, ...,    0,    0,    0],
       ...,
       [1652,  295,   30, ...,    0,    0,    0],
       [   2,  495,   30, ...,    0,    0,    0],
       [   4, 2114,  254, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input , Layer
from keras.layers import Embedding, GRU, Bidirectional , Reshape, Dot
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Input
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras import metrics
from keras import backend as K
from keras.regularizers import l2
import tensorflow as tf
from keras import initializers ,constraints, regularizers

In [ ]:
#implementation of attention layer in keras
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, CuDNNGRU
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#BiGRU+ATTENTION
sequence_input2 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences2 = embedding_layer(sequence_input2)
gru1=Bidirectional(CuDNNGRU(128, return_sequences=True))(embedded_sequences2) #bidirectional gru layer optimized for gpu
gru2= Bidirectional(CuDNNGRU(128, return_sequences=True))(gru1)
attn_outs=AttentionWithContext()(gru2) #attention layer
l_dense2 = Dense(128, activation='relu')(attn_outs)
preds2 = Dense(3, activation='softmax')(l_dense2) #ouput layer

model2 = Model(sequence_input2, preds2)
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("Bi-GRU+ATTENTION")
model2.summary()
cp2=ModelCheckpoint('model_gru_attn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
GRU
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          2850000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 256)          330240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300, 256)          296448    
_________________________________________________________________
attention_with_context_1 (At (None, 256)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
________________________________________

In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, Bidirectional
from keras.layers import Conv1D, MaxPooling1D, Embedding, CuDNNLSTM
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#BI-LSTM+ATTENTION
sequence_input3 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences3 = embedding_layer(sequence_input3)
lstm1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedded_sequences3) #bi-directional lstm layer optimized for gpu
lstm2= Bidirectional(CuDNNLSTM(128, return_sequences=True))(lstm1)
attn_outs3=AttentionWithContext()(lstm2) #attention layer
l_dense3 = Dense(128, activation='relu')(attn_outs3)
preds3 = Dense(3, activation='softmax')(l_dense3) #output layer

model3 = Model(sequence_input3, preds3)
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("Bi-LSTM+Attention")
model3.summary()
cp3=ModelCheckpoint('model_lstm_attn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

LSTM
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          2850000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 300, 256)          440320    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 300, 256)          395264    
_________________________________________________________________
attention_with_context_2 (At (None, 256)               66048     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                

In [ ]:
history2=model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp2]) #train bigru with attention with checkpoint


Train on 2034 samples, validate on 227 samples
Epoch 1/7
2034/2034 [==============================] - 11s 5ms/step - loss: 0.9161 - acc: 0.5929 - val_loss: 0.9273 - val_acc: 0.6211

Epoch 00001: val_acc improved from -inf to 0.62115, saving model to model_gru_attn.hdf5
Epoch 2/7
2034/2034 [==============================] - 4s 2ms/step - loss: 0.7007 - acc: 0.6991 - val_loss: 1.0517 - val_acc: 0.5947

Epoch 00002: val_acc did not improve from 0.62115
Epoch 3/7
2034/2034 [==============================] - 4s 2ms/step - loss: 0.4326 - acc: 0.8373 - val_loss: 1.2046 - val_acc: 0.5947

Epoch 00003: val_acc did not improve from 0.62115
Epoch 4/7
2034/2034 [==============================] - 4s 2ms/step - loss: 0.2345 - acc: 0.9174 - val_loss: 1.5691 - val_acc: 0.5066

Epoch 00004: val_acc did not improve from 0.62115
Epoch 5/7
2034/2034 [==============================] - 4s 2ms/step - loss: 0.1448 - acc: 0.9503 - val_loss: 1.8967 - val_acc: 0.5947

Epoch 00005: val_acc did not improve from 0

In [ ]:
history3=model3.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp3]) #train bilstm with attention with checkpoint

Train on 2034 samples, validate on 227 samples
Epoch 1/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.5714 - acc: 0.7748 - val_loss: 1.3842 - val_acc: 0.5815

Epoch 00001: val_acc improved from -inf to 0.58150, saving model to model_lstm_attn.hdf5
Epoch 2/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.1371 - acc: 0.9567 - val_loss: 1.3967 - val_acc: 0.5727

Epoch 00002: val_acc did not improve from 0.58150
Epoch 3/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.0788 - acc: 0.9813 - val_loss: 1.6993 - val_acc: 0.5507

Epoch 00003: val_acc did not improve from 0.58150
Epoch 4/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.0525 - acc: 0.9838 - val_loss: 2.1626 - val_acc: 0.5374

Epoch 00004: val_acc did not improve from 0.58150
Epoch 5/7
2034/2034 [==============================] - 6s 3ms/step - loss: 0.0305 - acc: 0.9926 - val_loss: 2.0397 - val_acc: 0.5551

Epoch 00005: val_acc did not improve from 0.

In [ ]:
w2=model2.predict(x_test) #predict on testing data for bi-gru with attention
w2=np.argmax(w2,axis=1)
w3=model3.predict(x_test) #predict on testing data for bi-lstm with attention
w3=np.argmax(w3,axis=1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
#true labels for validation data
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [ ]:
print("Bi-GRU+Attention")
print(classification_report(Y_actual,w2))
print("Bi-LSTM+Attention")
print(classification_report(Y_actual,w3))

GRU
              precision    recall  f1-score   support

           0       0.57      0.64      0.61        72
           1       0.11      0.05      0.07        39
           2       0.65      0.72      0.69       116

    accuracy                           0.58       227
   macro avg       0.45      0.47      0.45       227
weighted avg       0.53      0.58      0.55       227

LSTM
              precision    recall  f1-score   support

           0       0.57      0.72      0.63        72
           1       0.12      0.08      0.09        39
           2       0.67      0.64      0.65       116

    accuracy                           0.57       227
   macro avg       0.45      0.48      0.46       227
weighted avg       0.54      0.57      0.55       227



In [ ]:
#testing data
data_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/hasoc2019_en_test-2919_prepro.csv')
data_test=data_test[data_test.task_2!="NONE"] #remove rows where task2=NONE

In [ ]:
data_test.head()

,text_id,text,task_1,task_2,task_3
4,hasoc_en_568,fuck go back dark ages cow ibnliverealtime rap...,HOF,PRFN,UNT
9,hasoc_en_527,jesus christ christian news illuminati changin...,HOF,HATE,TIN
10,hasoc_en_137,even true muslims supporting zakir knkmow prom...,HOF,PRFN,TIN
11,hasoc_en_606,kerala halal course sickular bastion serve por...,HOF,HATE,TIN
12,hasoc_en_246,look sweetie it’s simple jain ask veg food get...,HOF,PRFN,TIN


In [ ]:
data_test.text=data_test.text.astype(str)
df_text=data_test['text']

In [ ]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)    #use the reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [ ]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_test = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300

In [ ]:
w_test2=model2.predict(X_test) #predict class of testing data for bi-gru+attn
w_test2=np.argmax(w_test2,axis=1)
w_test3=model3.predict(X_test)  #predict class of testing data for bi-lstm+attn
w_test3=np.argmax(w_test3,axis=1)

In [ ]:
data_test['task_2'] = data_test['task_2'].map({'PRFN':0, 'OFFN': 1,'HATE':2}) #map label strings to numbers

In [ ]:
y_test_actual=data_test['task_2']

In [ ]:
print("Bi-GRU+attention")
print(classification_report(y_test_actual,w_test2))
print("Bi-LSTM+attention")
print(classification_report(y_test_actual,w_test3))

GRU
              precision    recall  f1-score   support

           0       0.38      0.38      0.38        93
           1       0.22      0.11      0.15        71
           2       0.45      0.57      0.50       124

    accuracy                           0.40       288
   macro avg       0.35      0.35      0.34       288
weighted avg       0.37      0.40      0.37       288

LSTM
              precision    recall  f1-score   support

           0       0.45      0.52      0.48        93
           1       0.25      0.18      0.21        71
           2       0.47      0.49      0.48       124

    accuracy                           0.42       288
   macro avg       0.39      0.40      0.39       288
weighted avg       0.41      0.42      0.41       288

